初赛提供2012年1月-2017年10月盐城分车型销量配置数据，  参赛者允许使用任何可公开的外部数据辅助预测。 
初赛分为两个阶段，第一阶段需要参赛者预测2017年11月盐城分车型销量数据，第二阶段需要参赛者预测2017年12月盐城分车型销量数据。
第一阶段评测时间：2018年1月15日-2月25日
第二阶段评测时间：2018年2月26日-2月27日

数据时长：2012年1月 至 2017年10月

|                         |              |                                     |
|-------------------------|--------------|-------------------------------------|
| **字段**                | **含义**     | **示例**                            |
| sale\_date              | 销售日期     | 201201                              |
| class\_id               | 车型ID       | 234567                              |
| sale\_quantity          | 销量         | 15                                  |
| brand\_id               | 品牌ID       | 234                                 |
| compartment             | 厢数         | 3                                   |
| type\_id                | 车型类别ID   | 1                                   |
| level\_id               | 车型级别ID   | 1                                   |
| department\_id          | 车型系别ID   | 1                                   |
| TR                      | 变速器档位   | 6                                   |
| gearbox\_type           | 变速器形式   | AT                                  |
| displacement            | 排量         | 2.5                                 |
| if\_charging            | 是否增压     | L                                   |
| price\_level            | 成交段       | 35-50W（“W”：万元，“WL”：万元以下） |
| driven\_type\_id        | 驱动形式ID   | 1                                   |
| fuel\_type\_id          | 燃料种类ID   | 1                                   |
| newenergy\_type\_id     | 新能源类型ID | 1                                   |
| emission\_standards\_id | 排放标准ID   | 1                                   |
| if\_MPV\_id             | 是否微客MPV  | 1                                   |
| if\_luxurious\_id       | 是否豪华ID   | 1                                   |
| power                   | 功率         | 160                                 |
| cylinder\_number        | 缸数         | 6                                   |
| engine\_torque          | 发动机扭矩   | 250                                 |
| car\_length             | 车长         | 4531                                |
| car\_width              | 车宽         | 1817                                |
| car\_height             | 车高         | 1421                                |
| total\_quality          | 总质量       | 1980                                |
| equipment\_quality      | 整备质量     | 1565                                |
| rated\_passenger        | 额定载客     | 5                                   |
| wheelbase               | 轴距         | 2760                                |
| front\_track            | 前轮距       | 1500                                |
| rear\_track             | 后轮距       | 1529                                |


** 参赛者需提交的结果数据**

|**字段**   | **含义**   | **示例** |
|---------|-------------|-------- |
|predict_date      |预测日期   | 201711 |
|class_id | 车型ID | 123456 |
|predict_quantity | 预测销量 | 1234 |


|字段|sale_date|class_id|sale_quantity|brand_id|compartment|type_id|level_id|department_id|TR|gearbox_type|
|---|---------|--------|-------------|--------|-----------|-------|---------|------------|--|------------|
|含义|销售日期|车型ID|销量|品牌ID|厢数|车型类别ID|车型级别ID|车型系别ID|变速器档位|变速器形式|
|示例|201201|234567|15|234|3|1|1|1|6|AT|

|字段|displacement|if_charging|price_level|driven_type_id|fuel_type_id|newenergy_type_id|emission_standards_id|if_MPV_id|if_luxurious_id|power|
|---|---------|--------|-------------|--------|-----------|-------|---------|------------|--|------------|
|含义|排量|是否增压|成交段|驱动形式ID|燃料种类ID|新能源类型ID|排放标准ID|是否微客MPV|是否豪华ID|功率|
|示例|2.5|L|35-50W（“W”：万元，“WL”：万元以下）|1|1|1|1|1|1|160|

|字段|cylinder_number|engine_torque|car_length|car_width|car_height|total_quality|equipment_quality|rated_passenger|wheelbase|front_track|rear_track|
|---|---------|--------|-------------|--------|-----------|-------|---------|------------|--|------------|
|含义|缸数|发动机扭矩|车长|车宽|车高|总质量|整备质量|额定载客|轴距|前轮距|后轮距|
|示例|6|250|4531|1817|1421|1980|1565|5|2760|1500|1529|


**初赛结果评价指标**

主办方将依据参赛者提供的最终预测结果进行评价，评价指标的计算公式为：

$$ \varepsilon 
 = \sqrt{\frac{\Sigma_{i=1}^{n}{(p_i - a_i)^2}}{n}} $$
 
其中:

1. $\varepsilon$ 参赛者最终分数
2. $n$ 测试样本总量
3. $p_i$ 第$i$个测试样本的预测值
4. $a_i$ 第$i$个测试样本的真实值

In [97]:
%matplotlib inline
from __future__ import print_function
from __future__ import division
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import ElasticNetCV, LassoCV
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.neighbors import KNeighborsRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.svm import SVR
from sklearn.learning_curve import learning_curve
from sklearn.decomposition import PCA
from xgboost import XGBRegressor
from sklearn.preprocessing import LabelEncoder, LabelBinarizer
from sklearn.preprocessing import MinMaxScaler, RobustScaler
from scipy import stats
import pickle
import dateutil
pd.set_option('mode.chained_assignment',None)

In [268]:
def get_score(y_pred, y):
    return np.sqrt(np.mean((y_pred - y)**2))

def get_mean(x):
    """
    x like 7-8
    return int (7+8) /2 
    """
    if '-' in x:
        return round(np.mean([int(i) for i in x.split('-')]))
    else:
        return x

def get_car_level(res):
    """给汽车分级
    a) A00 级轿车：轴距在2～2.2 米之间，发动机排量小于1 升。
    b) A0 级轿车：轴距为2.2～2.3 米，排量为1～1.3 升。
    c) A 级车：轴距在2.3～2.45 米之间，排量约在1.3～1.6 升。
    d) B 级车：轴距约在2.45～2.6 米之间，排量从1.6～2.4 升。
    e) C 级轿车：轴距约在2.6～.8 米之间，发动机排量为2.4～3.0 升。
    f) D 级豪华轿车：大多外形气派，车内空间极为宽敞，发动机动力也非常强劲，
    """
    car_level = []
    for w,d in res.values:
        tmp = w * d / 1000
        if tmp < 2.2:
            car_level.append(0)
            continue 
        elif tmp < 2.99:
            car_level.append(1)
            continue
        elif tmp < 3.92:
            car_level.append(2)
            continue
        elif tmp < 6.24:
            car_level.append(3)
            continue
        elif tmp < 8.4:
            car_level.append(4)
            continue
        else:
            car_level.append(5)
    return car_level
    
def get_price(x,label):
    """
    15-25w
    low: 15
    high: 25
    5wl
    low:0
    high:5
    """
    x = x.lower()
    if '-' in x:
        x1, x2 = x.replace('w','').split('-')
    elif x.endswith('l'):
        x1 = 0
        x2 = x.replace('wl','')
    if label == 'low':
        return x1
    elif label == 'high':
        return x2
    elif label == 'mean':
        return (float(x1) + float(x2)) / 2

def eval_model(alg, train_data, y):
    print("eval model", alg)
    X_train, X_test, y_train, y_test = train_test_split(train_data, y, test_size=0.2, random_state=0)
    print("train shape", X_train.shape, "valid shape", X_test.shape)
    pred = alg.fit(X_train, y_train).predict(X_test)
    print("cv score:", get_score(pred, y_test))
    pred = alg.fit(train_data, y).predict(train_data)
    print("all score:", get_score(pred, y))
    return alg

def get_season(x):
    """根据日期返回季度
    1：春季，2:夏季，3：秋季，4：冬季
    """
    x = str(x)
    m = int(x[-2:])
    if m < 4:
        return 1
    elif m < 7:
        return 2
    elif m < 10:
        return 3
    else:
        return 4
def get_secs(x):
    """根据日期，返回从2012.1.1之后经历的秒数"""
    base_ts = dateutil.parser.parse('201201').timestamp()
    return dateutil.parse.parse(x).timestamp() - base_ts
    
def plot_pred_compare(pred_df):
    preview_df = pd.read_csv('./output/yancheng_mix-20180201_22.csv')
    plt.plot(np.arange(140), pred_df['predict_quantity'], label='curr predict')
    plt.plot(np.arange(140), preview_df['predict_quantity'], label='best predict')
    plt.legend(loc='best')
    plt.rcParams['figure.figsize'] = (18,6)
    plt.show()

In [3]:
df_train = pd.read_csv('./input/yancheng_train_20171226.csv',  low_memory=False)
df_test  = pd.read_csv('./input/yancheng_testA_20171225.csv',  low_memory=False)
y  = df_train['sale_quantity']
print("train shape:",df_train.shape)
#print("test shape:", df_test.shape)

train shape: (20157, 32)


In [ ]:
k = df_train[df_train.class_id == 321683]
subk = k[['sale_date','sale_quantity','gearbox_type','total_quality','equipment_quality','engine_torque']]
subg = subk.groupby(['gearbox_type','engine_torque','sale_date'])
subg.sale_quantity.sum()

每一个 class_id 其实还有细分，但是预测里只需要针对 class_id 进行预测。

因此考虑一个方法是找到每一个 class_id 里最大销量的那个型号，用这个型号作为该 class_id 的全部型号
然后把这个这个 class_id 下的每一月的销售和进行汇总。最后的训练集结果就是每一个 class_id 下每一月只有一条记录。

In [118]:
car_info = pd.DataFrame()
k = df_train[df_train.sale_date >201706].groupby(['class_id'])['sale_quantity'].max()
for class_id, max_sale_quantity in zip(k.index,k.values):
    #find the record
    res = df_train[df_train.class_id == class_id][df_train.sale_quantity == max_sale_quantity].iloc[:,3:]
    res.insert(0,'class_id', class_id)
    car_info = car_info.append(res, ignore_index=True)

/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:5: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """


In [119]:
len(df_train['class_id'].unique())

140

In [122]:
cdata = pd.DataFrame()
grp = df_train.groupby(['class_id','sale_date'])
sq = grp['sale_quantity'].sum()
for class_id, month in grp.groups.keys():
    res = car_info[car_info.class_id == class_id].copy()
    res.insert(1,'sale_quantity',sq[class_id, month])
    #res.insert(0,'class_id', class_id)
    res.insert(0,'sale_date', month)
    cdata = cdata.append(res)

In [123]:
cdata[cdata.class_id == 103507].head(2)

,sale_date,class_id,sale_quantity,brand_id,compartment,type_id,level_id,department_id,TR,gearbox_type,...,engine_torque,car_length,car_width,car_height,total_quality,equipment_quality,rated_passenger,wheelbase,front_track,rear_track
0,201503,103507,58,831,2,1,3,3,6,AT,...,150.7,4270,1780,1630,1830,1325,5,2590,1557,1570
0,201504,103507,232,831,2,1,3,3,6,AT,...,150.7,4270,1780,1630,1830,1325,5,2590,1557,1570


In [124]:
# 填补缺失的价格
cdata['price'][cdata.price == '-'] = cdata['price_level'][cdata.price == '-'].apply(lambda x: get_price(x, 'mean'))

In [125]:
cdata['TR'][cdata.TR.str.contains(';')] = cdata['TR'][cdata.TR.str.contains(';')].apply(lambda x: x.split(';')[0])
cdata['engine_torque'][cdata.engine_torque == '155/140'] = 155
cdata['power'][cdata.power == '81/70'] = 81
cdata['engine_torque'][cdata.engine_torque == '-'] = 0
cdata['level_id'][cdata.level_id == '-'] = 1
cdata['fuel_type_id'][cdata.fuel_type_id == '-'] = 1
cdata['rated_passenger'] = cdata.rated_passenger.apply(lambda x: get_mean(str(x)) )

In [127]:
cdata['TR'] = cdata['TR'].astype('int')
cdata['engine_torque'] = cdata['engine_torque'].astype('float')
cdata['power'] = cdata['power'].astype('float')
cdata['level_id'] = cdata['level_id'].astype('int')
cdata['fuel_type_id'] = cdata['fuel_type_id'].astype('int')
cdata['rated_passenger'] = cdata['rated_passenger'].astype('int')
cdata['price'] = cdata['price'].astype('float')

In [239]:
#把销售日期拆分成年和月
#cdata['year'] = cdata.sale_date.astype('str').apply(lambda x:x[:4]).astype('int')
#cdata['month'] = cdata['sale_date'].astype('str').apply(lambda x: x[-2:]).astype('int')
#cdata['ts'] = (cdata['year'] - 2012) * 12 + (cdata['month'] - 1)
#cdata['season'] = cdata['sale_date'].apply(lambda x: get_season(x))
#cdata['bc_id'] = cdata.brand_id.astype('str') + '_' + cdata.class_id.astype('str')
##计算车的体积，通过这个方式，RMSE 有所下降
cdata['car_vol'] = cdata.car_height * cdata.car_width * cdata.car_length / (1000*1000*1000)
## 依据轮距和排量，计算车辆分级
cdata['car_level'] = get_car_level(cdata[['wheelbase','displacement']])
cdata['space'] = cdata.wheelbase *  cdata.front_track * cdata.rear_track / (1000*1000*1000)
cdata.sample(10)

,sale_date,class_id,sale_quantity,brand_id,compartment,type_id,level_id,department_id,TR,gearbox_type,...,car_height,total_quality,equipment_quality,rated_passenger,wheelbase,front_track,rear_track,car_vol,space,car_level
164,201301,580634,487,692,2,2,1,5,0,CVT,...,1523,1640,1211,5,2700,1540,1540,11.512662,6.403320,3
155,201302,576298,1098,304,2,3,2,1,5,MT,...,1940,2080,1480,8,2800,1445,1420,15.274154,5.745320,3
81,201610,350259,340,750,3,2,1,4,7,DCT,...,1425,1825,1325,5,2656,1580,1554,11.964226,6.521330,2
82,201311,354068,635,692,3,2,1,5,0,CVT,...,1503,1650,1236,5,2700,1540,1540,12.250292,6.403320,3
17,201407,178529,314,98,1,4,1,1,5,MT,...,1820,1575,965,7,2500,1290,1290,10.434915,4.160250,2
275,201502,963845,221,836,2,1,3,3,6,AT,...,1622,1752,1341,5,2590,1557,1570,12.328173,6.331229,3
187,201703,628041,614,750,2,2,1,4,6,AT,...,1452,1740,1260,5,2637,1549,1520,11.114690,6.208764,3
41,201704,249875,190,294,2,1,1,6,0,CVT,...,1678,2004,1557,5,2705,1597,1589,14.448412,6.864297,3
70,201706,302513,143,864,2,1,3,1,5,MT,...,1654,1670,1285,5,2610,1585,1586,13.250789,6.561044,2
13,201401,175962,680,750,3,2,1,4,6,AT,...,1469,1640,1160,5,2603,1460,1500,11.209848,5.700570,3


In [251]:
len(cdata.class_id[cdata.sale_date > 201706].unique())

140

In [270]:
lb = LabelEncoder()
#lb = LabelBinarizer()
scaler = MinMaxScaler()
y = cdata['sale_quantity'].astype('int')
train_data = cdata.copy()
# unused_features = ['TR', 'emission_standards_id', 'displacement', 'gearbox_type', 'level_id', 'compartment', 
#                    'type_id', 'if_charging', 'rated_passenger', 'if_MPV_id', 'driven_type_id', 
#                    'if_luxurious_id', 'cylinder_number', 'newenergy_type_id', 'fuel_type_id','sale_quantity',
#                   'sale_date','year','month','price']
unused_features = ['sale_quantity','car_width','car_height','car_length','rated_passenger','wheelbase',
                   'front_track','rear_track']
train_data.drop(unused_features,axis=1, inplace=True)
logy = np.log1p(y)
#对特征数据进行处理：数值型进行标准化，条目型进行条目编码
for feature in  train_data.columns:
    if train_data[feature].dtype == 'object':
        train_data[feature] = lb.fit_transform(train_data[feature])        
    #else:
    #    train_data[feature] = scaler.fit_transform(train_data[feature].values.reshape(-1,1))
print("train shape:", train_data.shape)
print("lable shape:", y.shape)

train shape: (13194, 27)
lable shape: (13194,)


In [267]:
gdm = XGBRegressor(learning_rate=0.1, n_estimators=3000, n_jobs=-1, nthread=32,
              seed=0, max_depth=7,gamma=7.0, subsample=0.7, base_score=0.8)
lr = LinearRegression(n_jobs=-1)
knn = KNeighborsRegressor(algorithm='auto', leaf_size=20, metric='minkowski',
          metric_params=None, n_jobs=-1, n_neighbors=5, p=2,
          weights='distance')

In [271]:
gdm = eval_model(gdm, train_data, y)

eval model XGBRegressor(base_score=0.8, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=7.0, learning_rate=0.1, max_delta_step=0,
       max_depth=7, min_child_weight=1, missing=None, n_estimators=3000,
       n_jobs=-1, nthread=32, objective='reg:linear', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=0, silent=True,
       subsample=0.7)
train shape (10555, 27) valid shape (2639, 27)
cv score: 118.87431508658311
all score: 3.657116303868535


In [272]:
fi = pd.DataFrame({'name':train_data.columns, 'importance': gdm.feature_importances_})
fi.sort_values(['importance'], axis=0, ascending=False, inplace=True)
fi

,importance,name
0,0.559900,sale_date
1,0.095184,class_id
12,0.057175,price
2,0.035061,brand_id
24,0.034888,car_vol
25,0.033993,space
22,0.031560,total_quality
23,0.025357,equipment_quality
19,0.025030,power
21,0.018467,engine_torque


In [95]:
if False:
    pickle.dump(cdata, open('./output/train_data.bin','wb'))

In [ ]:
from datetime import datetime
plt.gca().xaxis.set_major_formatter(mdates.DateFormatter('%Y%m'))
plt.gca().xaxis.set_major_locator(mdates.MonthLocator())
plt.gcf().autofmt_xdate() 
data = cdata[['sale_date','class_id','sale_quantity']].sort_values('sale_date', axis=0, ascending=True)
miss_data_classid = []
plt.rcParams['figure.figsize'] = (18,20)
for class_id in [103507]:
    quantity = data.sale_quantity[data.class_id == class_id]
    xs = [datetime.strptime(d, '%Y%m').date() for d in data.sale_date[data.class_id == class_id].astype('str').unique()]
    plt.plot(xs, quantity, label=class_id)
plt.legend(loc='best')
plt.ylim(10, 5000)
plt.show()

In [275]:
import time
test_data = pd.DataFrame()
for class_id in df_test.class_id.unique():
    res = train_data[train_data.class_id == class_id].iloc[0,:]
    res['class_id'] = int(class_id)
    res['sale_date'] = 201711
    test_data = test_data.append(res)
test_data = test_data.reindex(train_data.columns, axis=1)
pred = gdm.predict(test_data)

In [ ]:
pred_result = pd.DataFrame({'predict_date': df_test.predict_date, 
                           'class_id': df_test.class_id, 'predict_quantity':np.abs(pred.astype('int'))})
today = time.strftime('%Y%m%d_%H')
pred_result = pred_result.reindex(['predict_date','class_id','predict_quantity'], axis=1)
#pred_result.to_csv('./output/yancheng-{}.csv'.format(today), index=False)
#pred_result.head(10)
plot_pred_compare(pred_result)